In [1]:
import numpy as np  
import pandas as pd 
import os, csv, sys
import glob
import subprocess
from tqdm.notebook import tqdm
from dataclasses import dataclass
from scipy.interpolate import InterpolatedUnivariateSpline
from datetime import datetime, timezone, timedelta

folder_path = os.path.abspath('gnss_analysis')
sys.path.append(folder_path)
from gnssutils import EphemerisManager

ModuleNotFoundError: No module named 'numpy'

Data

In [27]:
path = 'gnss_2024_06_01_12_00_xm.csv'
gnss = pd.read_csv(path, index_col=False)
selected_col = ['utcTimeMillis','Svid','ConstellationType','TimeNanos','FullBiasNanos','ReceivedSvTimeNanos','ReceivedSvTimeUncertaintyNanos','TimeOffsetNanos']
filtered_gnss = gnss[gnss['ReceivedSvTimeNanos'] > 1e14]
measurement = filtered_gnss[selected_col]
measurement

,utcTimeMillis,Svid,ConstellationType,TimeNanos,FullBiasNanos,ReceivedSvTimeNanos,ReceivedSvTimeUncertaintyNanos,TimeOffsetNanos
71,1717218052000,30,5,155835522000000,-9223372036854775808,5.364557e+14,12.0,0.0
72,1717218052000,29,5,155835522000000,-9223372036854775808,5.364557e+14,14.0,0.0
73,1717218052000,27,5,155835522000000,-9223372036854775808,5.364557e+14,40.0,0.0
74,1717218052000,20,5,155835522000000,-9223372036854775808,5.364557e+14,40.0,0.0
75,1717218052000,13,5,155835522000000,-9223372036854775808,5.364557e+14,13.0,0.0
...,...,...,...,...,...,...,...,...
6301,1717218264000,7,6,156048122000000,-1401097434289769506,5.366823e+14,53.0,0.0
6302,1717218264000,21,6,156048122000000,-1401097434289769506,5.366823e+14,40.0,0.0
6303,1717218264000,26,6,156048122000000,-1401097434289769506,5.366823e+14,9.0,0.0
6304,1717218264000,31,6,156048122000000,-1401097434289769506,5.366823e+14,92.0,0.0


In [28]:
# Lọc các vệ tinh thuộc GPS (ConstellationType == 1)
measurement.loc[measurement['ConstellationType'] == 1, 'Constellation'] = 'G'
# Chỉ lấy các vệ tinh bắt GPS
measurement = measurement.loc[measurement['Constellation'] == 'G']
# Đảm bảo cột 'Svid' là chuỗi ký tự để có thể sử dụng hàm str.len()
measurement['Svid'] = measurement['Svid'].astype(str)
# Thêm số 0 vào trước 'Svid' nếu chiều dài của nó là 1
measurement.loc[measurement['Svid'].str.len() == 1, 'Svid'] = '0' + measurement['Svid']
# Tạo cột mới 'SvName' bằng cách nối 'Constellation' và 'Svid'
measurement['SvName'] = measurement['Constellation'] + measurement['Svid']

C:\Users\Admin\AppData\Local\Temp\ipykernel_14736\3452372983.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  measurement.loc[measurement['ConstellationType'] == 1, 'Constellation'] = 'G'
C:\Users\Admin\AppData\Local\Temp\ipykernel_14736\3452372983.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  measurement['Svid'] = measurement['Svid'].astype(str)
C:\Users\Admin\AppData\Local\Temp\ipykernel_14736\3452372983.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [29]:
# measurement['GpsTimeNanos'] = measurement['TimeNanos'] - measurement['FullBiasNanos']
# gpsepoch = datetime(1980, 1, 6, 0, 0, 0)
# measurement['UnixTime'] = pd.to_datetime(measurement['GpsTimeNanos'], utc = True, origin=gpsepoch)
# measurement['UnixTime'] = measurement['UnixTime']

# # Split data into measurement epochs
# measurement['Epoch'] = 0
# measurement.loc[measurement['UnixTime'] - measurement['UnixTime'].shift() > timedelta(milliseconds=200), 'Epoch'] = 1
# measurement['Epoch'] = measurement['Epoch'].cumsum()
# measurement

In [30]:
WEEKSEC = 604800
LIGHTSPEED = 2.99792458e8

# This should account for rollovers since it uses a week number specific to each measurement

measurement['tRxGnssNanos'] = measurement['TimeNanos'] + measurement['TimeOffsetNanos'] - (measurement['FullBiasNanos'].iloc[0])
measurement['GpsWeekNumber'] = np.floor(1e-9 * measurement['tRxGnssNanos'] / WEEKSEC)
measurement['tRxSeconds'] = 1e-9*measurement['tRxGnssNanos'] - WEEKSEC * measurement['GpsWeekNumber']
measurement['tTxSeconds'] = 1e-9*(measurement['ReceivedSvTimeNanos'] + measurement['TimeOffsetNanos'])
# Calculate pseudorange in seconds
measurement['prSeconds'] = measurement['tRxSeconds'] - measurement['tTxSeconds']

# Conver to meters
measurement['PrM'] = LIGHTSPEED * measurement['prSeconds']
measurement['PrSigmaM'] = LIGHTSPEED * 1e-9 * measurement['ReceivedSvTimeUncertaintyNanos']


C:\Users\Admin\AppData\Local\Temp\ipykernel_14736\1796814486.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  measurement['tRxGnssNanos'] = measurement['TimeNanos'] + measurement['TimeOffsetNanos'] - (measurement['FullBiasNanos'].iloc[0])
C:\Users\Admin\AppData\Local\Temp\ipykernel_14736\1796814486.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  measurement['GpsWeekNumber'] = np.floor(1e-9 * measurement['tRxGnssNanos'] / WEEKSEC)
C:\Users\Admin\AppData\Local\Temp\ipykernel_14736\1796814486.py:8: Sett

In [31]:
#chuyển giá trị thời gian về dạng date time
measurement['UnixTime'] = pd.to_datetime(measurement['utcTimeMillis'],unit='ms', utc=True)
#Chia measurement thành các epoch, mỗi epoch gồm các vệ tinh có cùng thời gian thu
measurement['Epoch'] = 0
measurement.loc[measurement['UnixTime'] - measurement['UnixTime'].shift() > timedelta(milliseconds=200), 'Epoch'] = 1
measurement['Epoch'] = measurement['Epoch'].cumsum()
measurement

C:\Users\Admin\AppData\Local\Temp\ipykernel_14736\42048977.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  measurement['UnixTime'] = pd.to_datetime(measurement['utcTimeMillis'],unit='ms', utc=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_14736\42048977.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  measurement['Epoch'] = 0
C:\Users\Admin\AppData\Local\Temp\ipykernel_14736\42048977.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

,utcTimeMillis,Svid,ConstellationType,TimeNanos,FullBiasNanos,ReceivedSvTimeNanos,ReceivedSvTimeUncertaintyNanos,TimeOffsetNanos,Constellation,SvName,tRxGnssNanos,GpsWeekNumber,tRxSeconds,tTxSeconds,prSeconds,PrM,PrSigmaM,UnixTime,Epoch
143,1717218055000,05,1,155839302000000,-1401097434289768911,5.364735e+14,21.0,0.0,G,G05,1.401253e+18,2316.0,536473.591769,536473.514190,0.077579,2.325752e+07,6.295642,2024-06-01 05:00:55+00:00,0
144,1717218055000,06,1,155839302000000,-1401097434289768911,5.364735e+14,12.0,0.0,G,G06,1.401253e+18,2316.0,536473.591769,536473.518932,0.072837,2.183606e+07,3.597509,2024-06-01 05:00:55+00:00,0
145,1717218055000,09,1,155839302000000,-1401097434289768911,5.364735e+14,31.0,0.0,G,G09,1.401253e+18,2316.0,536473.591769,536473.513300,0.078470,2.352462e+07,9.293566,2024-06-01 05:00:55+00:00,0
146,1717218055000,11,1,155839302000000,-1401097434289768911,5.364735e+14,7.0,0.0,G,G11,1.401253e+18,2316.0,536473.591769,536473.519098,0.072671,2.178617e+07,2.098547,2024-06-01 05:00:55+00:00,0
147,1717218055000,12,1,155839302000000,-1401097434289768911,5.364735e+14,12.0,0.0,G,G12,1.401253e+18,2316.0,536473.591769,536473.515246,0.076523,2.294096e+07,3.597509,2024-06-01 05:00:55+00:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6279,1717218264000,12,1,156048122000000,-1401097434289769506,5.366823e+14,31.0,0.0,G,G12,1.401253e+18,2316.0,536682.411769,536682.335562,0.076207,2.284614e+07,9.293566,2024-06-01 05:04:24+00:00,209
6280,1717218264000,14,1,156048122000000,-1401097434289769506,5.366823e+14,40.0,0.0,G,G14,1.401253e+18,2316.0,536682.411769,536682.331230,0.080538,2.414483e+07,11.991698,2024-06-01 05:04:24+00:00,209
6281,1717218264000,17,1,156048122000000,-1401097434289769506,5.366823e+14,11.0,0.0,G,G17,1.401253e+18,2316.0,536682.411769,536682.337792,0.073977,2.217780e+07,3.297717,2024-06-01 05:04:24+00:00,209
6282,1717218264000,19,1,156048122000000,-1401097434289769506,5.366823e+14,31.0,0.0,G,G19,1.401253e+18,2316.0,536682.411769,536682.342461,0.069308,2.077792e+07,9.293566,2024-06-01 05:04:24+00:00,209


In [32]:
#lấy dữ liệu ephemeris
ephemeris_data_directory = 'output'
manager = EphemerisManager(ephemeris_data_directory)

epoch = 0
num_sat = 0

while num_sat < 4:
    one_epoch = measurement.loc[measurement['Epoch'] == epoch].drop_duplicates(subset='SvName')
    
    if not one_epoch.empty:
        timestamp = pd.to_datetime(one_epoch.iloc[0]['UnixTime'], unit='s')
        one_epoch.set_index('SvName', inplace=True)
        num_sat = len(one_epoch.index)
    
    epoch += 1

sats = one_epoch.index.unique().tolist()
ephemeris = manager.get_ephemeris(timestamp, sats)
one_epoch

,utcTimeMillis,Svid,ConstellationType,TimeNanos,FullBiasNanos,ReceivedSvTimeNanos,ReceivedSvTimeUncertaintyNanos,TimeOffsetNanos,Constellation,tRxGnssNanos,GpsWeekNumber,tRxSeconds,tTxSeconds,prSeconds,PrM,PrSigmaM,UnixTime,Epoch
SvName,,,,,,,,,,,,,,,,,,
G05,1717218055000,05,1,155839302000000,-1401097434289768911,5.364735e+14,21.0,0.0,G,1.401253e+18,2316.0,536473.591769,536473.514190,0.077579,2.325752e+07,6.295642,2024-06-01 05:00:55+00:00,0
G06,1717218055000,06,1,155839302000000,-1401097434289768911,5.364735e+14,12.0,0.0,G,1.401253e+18,2316.0,536473.591769,536473.518932,0.072837,2.183606e+07,3.597509,2024-06-01 05:00:55+00:00,0
G09,1717218055000,09,1,155839302000000,-1401097434289768911,5.364735e+14,31.0,0.0,G,1.401253e+18,2316.0,536473.591769,536473.513300,0.078470,2.352462e+07,9.293566,2024-06-01 05:00:55+00:00,0
G11,1717218055000,11,1,155839302000000,-1401097434289768911,5.364735e+14,7.0,0.0,G,1.401253e+18,2316.0,536473.591769,536473.519098,0.072671,2.178617e+07,2.098547,2024-06-01 05:00:55+00:00,0
G12,1717218055000,12,1,155839302000000,-1401097434289768911,5.364735e+14,12.0,0.0,G,1.401253e+18,2316.0,536473.591769,536473.515246,0.076523,2.294096e+07,3.597509,2024-06-01 05:00:55+00:00,0
G14,1717218055000,14,1,155839302000000,-1401097434289768911,5.364735e+14,31.0,0.0,G,1.401253e+18,2316.0,536473.591769,536473.511715,0.080054,2.399962e+07,9.293566,2024-06-01 05:00:55+00:00,0
G17,1717218055000,17,1,155839302000000,-1401097434289768911,5.364735e+14,35.0,0.0,G,1.401253e+18,2316.0,536473.591769,536473.517924,0.073845,2.213826e+07,10.492736,2024-06-01 05:00:55+00:00,0
G19,1717218055000,19,1,155839302000000,-1401097434289768911,5.364735e+14,24.0,0.0,G,1.401253e+18,2316.0,536473.591769,536473.522483,0.069286,2.077153e+07,7.195019,2024-06-01 05:00:55+00:00,0
G20,1717218055000,20,1,155839302000000,-1401097434289768911,5.364735e+14,21.0,0.0,G,1.401253e+18,2316.0,536473.591769,536473.521580,0.070190,2.104229e+07,6.295642,2024-06-01 05:00:55+00:00,0


In [33]:
def calculate_satellite_position(ephemeris, transmit_time):
    mu = 3.986005e14
    OmegaDot_e = 7.2921151467e-5
    F = -4.442807633e-10
    sv_position = pd.DataFrame()
    sv_position['sv']= ephemeris.index
    sv_position.set_index('sv', inplace=True)
    sv_position['t_k'] = transmit_time - ephemeris['t_oe']
    A = ephemeris['sqrtA'].pow(2)
    n_0 = np.sqrt(mu / A.pow(3))
    n = n_0 + ephemeris['deltaN']
    M_k = ephemeris['M_0'] + n * sv_position['t_k']
    E_k = M_k
    err = pd.Series(data=[1]*len(sv_position.index))
    i = 0
    while err.abs().min() > 1e-8 and i < 10:
        new_vals = M_k + ephemeris['e']*np.sin(E_k)
        err = new_vals - E_k
        E_k = new_vals
        i += 1
        
    sinE_k = np.sin(E_k)
    cosE_k = np.cos(E_k)
    delT_r = F * ephemeris['e'].pow(ephemeris['sqrtA']) * sinE_k
    delT_oc = transmit_time - ephemeris['t_oc']
    sv_position['delT_sv'] = ephemeris['SVclockBias'] + ephemeris['SVclockDrift'] * delT_oc + ephemeris['SVclockDriftRate'] * delT_oc.pow(2)

    v_k = np.arctan2(np.sqrt(1-ephemeris['e'].pow(2))*sinE_k,(cosE_k - ephemeris['e']))

    Phi_k = v_k + ephemeris['omega']

    sin2Phi_k = np.sin(2*Phi_k)
    cos2Phi_k = np.cos(2*Phi_k)

    du_k = ephemeris['C_us']*sin2Phi_k + ephemeris['C_uc']*cos2Phi_k
    dr_k = ephemeris['C_rs']*sin2Phi_k + ephemeris['C_rc']*cos2Phi_k
    di_k = ephemeris['C_is']*sin2Phi_k + ephemeris['C_ic']*cos2Phi_k

    u_k = Phi_k + du_k

    r_k = A*(1 - ephemeris['e']*np.cos(E_k)) + dr_k

    i_k = ephemeris['i_0'] + di_k + ephemeris['IDOT']*sv_position['t_k']

    x_k_prime = r_k*np.cos(u_k)
    y_k_prime = r_k*np.sin(u_k)

    Omega_k = ephemeris['Omega_0'] + (ephemeris['OmegaDot'] - OmegaDot_e)*sv_position['t_k'] - OmegaDot_e*ephemeris['t_oe']

    sv_position['x_k'] = x_k_prime*np.cos(Omega_k) - y_k_prime*np.cos(i_k)*np.sin(Omega_k)
    sv_position['y_k'] = x_k_prime*np.sin(Omega_k) + y_k_prime*np.cos(i_k)*np.cos(Omega_k)
    sv_position['z_k'] = y_k_prime*np.sin(i_k)
    return sv_position

# Run the function and check out the results:
sv_position = calculate_satellite_position(ephemeris, one_epoch['tTxSeconds'])
print(sv_position)

             t_k   delT_sv           x_k           y_k           z_k
sv                                                                  
G05  3673.514190 -0.000175  6.136833e+06  2.389080e+07 -9.910740e+06
G06  3673.518932  0.000243 -7.683355e+06  1.244575e+07  2.222718e+07
G09  3673.513300  0.000218 -2.456996e+07  6.538651e+06  7.694324e+06
G11  3673.519098 -0.000676  5.619935e+06  1.929543e+07  1.741586e+07
G12  3673.515246 -0.000510  1.118836e+07  1.459532e+07  1.889999e+07
G14  3673.511715  0.000415 -1.320274e+07  1.645083e+07 -1.597681e+07
G17  3689.517924  0.000700 -2.217486e+07  1.363012e+07  6.117261e+06
G19  3673.522483  0.000490 -1.602253e+07  1.574877e+07  1.367178e+07
G20  3673.521580  0.000375  1.931277e+06  2.655191e+07  1.118846e+06
G22  3673.517830 -0.000021 -1.007453e+07  2.291824e+07 -8.875482e+06


In [34]:
#initial guesses of receiver clock bias and position
b0 = 0
x0 = np.array([0, 0, 0])
xs = sv_position[['x_k', 'y_k', 'z_k']].to_numpy()

# Apply satellite clock bias to correct the measured pseudorange values
pr = one_epoch['PrM'] + LIGHTSPEED * sv_position['delT_sv']
pr = pr.to_numpy()

In [35]:
def my_ecef2lla(pos):
  # x, y and z are scalars or vectors in meters
  x = pos[0]
  y = pos[1]
  z = pos[2]
  a=6378137
  a_sq=a**2
  e = 8.181919084261345e-2
  e_sq = 6.69437999014e-3

  f = 1/298.257223563
  b = a*(1-f)

  # calculations:
  r = np.sqrt(x**2 + y**2)
  ep_sq  = (a**2-b**2)/b**2
  ee = (a**2-b**2)
  f = (54*b**2)*(z**2)
  g = r**2 + (1 - e_sq)*(z**2) - e_sq*ee*2
  c = (e_sq**2)*f*r**2/(g**3)
  s = (1 + c + np.sqrt(c**2 + 2*c))**(1/3.)
  p = f/(3.*(g**2)*(s + (1./s) + 1)**2)
  q = np.sqrt(1 + 2*p*e_sq**2)
  r_0 = -(p*e_sq*r)/(1+q) + np.sqrt(0.5*(a**2)*(1+(1./q)) - p*(z**2)*(1-e_sq)/(q*(1+q)) - 0.5*p*(r**2))
  u = np.sqrt((r - e_sq*r_0)**2 + z**2)
  v = np.sqrt((r - e_sq*r_0)**2 + (1 - e_sq)*z**2)
  z_0 = (b**2)*z/(a*v)
  h = u*(1 - b**2/(a*v))
  phi = np.arctan((z + ep_sq*z_0)/r)
  lambd = np.arctan2(y, x)

  return phi*180/np.pi, lambd*180/np.pi, h

In [36]:
omegaE = 7.2921151467e-5
def least_squares(xs, measured_pseudorange, x0, b0):
    dx = 100*np.ones(3)
    b = b0
    # set up the G matrix with the right dimensions. We will later replace the first 3 columns
    # note that b here is the clock bias in meters equivalent, so the actual clock bias is b/LIGHTSPEED
    G = np.ones((measured_pseudorange.size, 4))
    iterations = 0
    while np.linalg.norm(dx) > 1e-3:
        # Eq. (2):
        r = np.linalg.norm(xs - x0, axis=1)
        # Eq. (1):
        phat = r + b0
        # Eq. (3):
        deltaP = measured_pseudorange - phat
        G[:, 0:3] = -(xs - x0) / r[:, None]
        # Eq. (4):
        sol = np.linalg.inv(np.transpose(G) @ G) @ np.transpose(G) @ deltaP
        # Eq. (5):
        dx = sol[0:3]
        db = sol[3]
        x0 = x0 + dx
        b0 = b0 + db
    norm_dp = np.linalg.norm(deltaP)
    return x0, b0, norm_dp

x, b, dp = least_squares(xs, pr, x0, b0)

print(my_ecef2lla(x))
print(b/LIGHTSPEED)
print(dp)

(21.005072329864852, 105.84163251781591, 50.167144253565965)
5.050062156407327e-07
39.46206046324121


In [37]:
ecef_list = []
for epoch in measurement['Epoch'].unique():
    one_epoch = measurement.loc[(measurement['Epoch'] == epoch) & (measurement['prSeconds'] < 0.1)] 
    one_epoch = one_epoch.drop_duplicates(subset='SvName').set_index('SvName')
    if len(one_epoch.index) > 4:
        timestamp = one_epoch.iloc[0]['UnixTime'].to_pydatetime(warn=False)
        sats = one_epoch.index.unique().tolist()
        ephemeris = manager.get_ephemeris(timestamp, sats)
        sv_position = calculate_satellite_position(ephemeris, one_epoch['tTxSeconds'])

        xs = sv_position[['x_k', 'y_k', 'z_k']].to_numpy()
        pr = one_epoch['PrM'] + LIGHTSPEED * sv_position['delT_sv']
        pr = pr.to_numpy()

        x, b, dp = least_squares(xs, pr, x, b)
        ecef_list.append(x)

In [ ]:
import navpy
ecef_array = np.stack(ecef_list, axis=0)
lla_array = np.stack(navpy.ecef2lla(ecef_array), axis=1)

# Convert back to Pandas and save to csv
lla_df = pd.DataFrame(lla_array, columns=['Latitude', 'Longitude', 'Altitude'])
lla_df.to_csv('calculated_postion.csv')
